In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
alphabets = list("ABCDEFGHIJ")

In [ ]:
images = []
for alphabet in alphabets:
    for img in os.listdir(f"../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/{alphabet}"):
        image = cv2.imread(f"../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train/{alphabet}/{img}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (100,100))
        image = image/255.0
        images.append(image.reshape((100,100,1)))

In [ ]:
images = np.asarray(images)
images.shape

In [ ]:
labels = []
for i in range(len(alphabets)):
    labels+=([i]*3000)

In [ ]:
labels = np.asarray(labels)
labels.shape

In [ ]:
from keras.utils import to_categorical
labels = to_categorical(labels, 10)
labels[0]

In [ ]:
from sklearn.utils import shuffle
images, labels = shuffle(images, labels)

plt.figure(figsize = (15,15))
print([np.argmax(i) for i in labels[:25]])
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(images[i].reshape((100,100)), cmap = "gray")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, BatchNormalization

model = Sequential()
model.add(Conv2D(128, (3,3), activation='relu', input_shape = X_train.shape[1:]))
model.add(MaxPool2D())
model.add(Conv2D(256, (3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(512, (3,3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, validation_data=(X_test, y_test), epochs=3)

In [ ]:
print(np.argmax(y_test[583]))
print(np.argmax(model.predict(X_test[583].reshape((1,100,100,1)))))

In [ ]:
model_json = model.to_json()
with open("asl_10.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
model.save_weights("asl_10_weights.h5")